# Getting Started - Create & Run Application

Welcome to the application creation phase of the PhariaAI tutorial! This guide will help you set up your environment to build and run custom applications with PhariaAI.

## What you'll learn

This tutorial will show you how to:
- Activate your existing virtual environment from the previous tutorial
- Install additional dependencies needed for application development
- Run terminal commands to build and deploy your PhariaAI applications

# The Core Concepts

<details>
<summary><strong>📚 Click to expand Core Concepts section</strong></summary>

<br>

> **Note:** This section provides a quick mental model of how PhariaKernel components work together. Detailed implementation steps follow in the next notebooks.

## Quick Overview

PhariaKernel lets you deploy AI components as serverless functions. Here's the workflow:

1. **Write** a Skill (Python function with AI logic)
2. **Build** it to WebAssembly 
3. **Publish** to a registry
4. **Configure** which Skills are available
5. **Invoke** via REST API

<div align="center">
  <img src="./images/skill_flow.png" width="600" alt="PhariaKernel Deployment Workflow">
  <p><em>Complete deployment workflow from development to production</em></p>
</div>

---

## Concept 1: The PhariaKernel Instance

**What:** A serverless runtime specifically designed for AI workloads - think AWS Lambda, but optimized for AI.

**Why it matters:** Handles all infrastructure complexity (scaling, queuing, load balancing) so you focus only on AI logic.

**To connect:** You need two environment variables from your PhariaKernel operator:
```bash
PHARIA_KERNEL_ADDRESS=<your-instance-url>
PHARIA_AI_TOKEN=<your-auth-token>
```

---

## Concept 2: Skills

**What:** Python functions that encapsulate AI logic with defined input/output schemas using Pydantic models.

**Why it matters:** Skills are stateless, strongly-typed, and testable - perfect for production AI systems.

<div align="center">
  <img src="./images/skill_structure.png" width="400" alt="Skill Structure">
  <p><em>Anatomy of a Skill: Input Schema → Business Logic → Output Schema</em></p>
</div>

**Example:**
```python
from pharia_skill import ChatParams, Csi, Message, skill
from pydantic import BaseModel

class Input(BaseModel):
    topic: str

class Output(BaseModel):
    haiku: str

@skill
def create_haiku(csi: Csi, input: Input) -> Output:
    response = csi.chat(
        "llama-3.1-8b-instruct",
        [Message.user(f"Write a haiku about {input.topic}")],
        ChatParams(max_tokens=64)
    )
    return Output(haiku=response.message.content)
```

---

## Concept 3: Building Skills

**What:** Compile your Python Skill to WebAssembly for secure, portable execution.

**Why it matters:** WASM provides isolation, consistent performance, and language independence.

<div align="center">
  <img src="./images/skill_build.png" width="500" alt="Build Process">
  <p><em>Build process transforms Python code into portable WebAssembly</em></p>
</div>

**Command:**
```bash
uv run pharia-skill build my_skill
```
This creates `my_skill.wasm` ready for deployment.

---

## Concept 4: Skill Registry

**What:** A container registry (like Docker Hub) but for WASM components.

**Why it matters:** Central storage for versioned Skills that PhariaKernel can load on-demand.

<div align="center">
  <img src="./images/skill_registry.png" width="450" alt="Skill Registry">
  <p><em>OCI-compliant registries store and version your Skills</em></p>
</div>

**Setup:** Configure registry credentials (get from your DevOps team):
```bash
export SKILL_REGISTRY=registry.example.com      # Registry URL
export SKILL_REPOSITORY=team/skills             # Your namespace
export SKILL_REGISTRY_USER=<username>           # Your username  
export SKILL_REGISTRY_TOKEN=<token>             # Access token
```

**Publish:**
```bash
uv run pharia-skill publish my_skill.wasm --name my_skill
```

---

## Concept 5: Namespace Configuration

**What:** A TOML file that declares which Skills should be available in your namespace.

**Why it matters:** Controls what Skills are loaded and accessible via API.

<div align="center">
  <img src="./images/skill_namespace.png" width="500" alt="Namespace Configuration">
  <p><em>GitOps workflow: Configuration changes trigger automatic Skill deployment</em></p>
</div>

**Example `namespace.toml`:**
```toml
skills = [
    { name = "my_skill", tag = "latest" },
    { name = "another_skill", tag = "v1.0.0" }
]
```

**Update process:** 
1. Edit namespace.toml in your Git repo
2. Create pull request
3. Merge → PhariaKernel auto-loads new Skills

---

## Concept 6: Invoking Skills

**What:** Call your deployed Skills via REST API.

**Example:**
```bash
curl 'https://pharia-kernel.domain/v1/skills/{namespace}/{skill_name}/run' \
  --header "Authorization: Bearer $PHARIA_AI_TOKEN" \
  --header 'Content-Type: application/json' \
  --data '{"topic": "mountains"}'
```

**Response:**
```json
{
  "haiku": "Peaks touch the sky high\nSnow-capped giants stand in silence\nClouds dance at their feet"
}
```

---

## 🎯 Key Takeaway

PhariaKernel transforms your Python AI functions into production-ready, scalable services with just a few commands. The complexity is hidden - you write Python, PhariaKernel handles the rest.

> **Need Details?** Contact your PhariaKernel operator for environment-specific setup. They'll provide connection details, registry access, and namespace configuration.

</details>

---

## Prerequisites and Setup

### Accessing Your Virtual Environment

If you completed the previous [**"0. Getting Started - Document Search Setup"**](../1.%20Enable%20searching%20documents/0.%20Getting%20Started.ipynb) tutorial, you already have a virtual environment set up. Simply activate it:

**On macOS/Linux:**
```python
source .venv/bin/activate
```

**On Windows:**
```PowerShell
.venv\Scripts\activate
```

---

## Required Software Installation

You'll need additional tools for application development and containerization.

### 1. Installing Node.js and npx

**macOS Installation:**

1. **Install Node.js using Homebrew** (if not already installed):
   ```bash
   brew install node
   ```
   This automatically includes `npx`.

2. **Verify installation:**
   ```bash
   node --version
   npx --version
   ```

**Windows Installation:**

1. **Install Node.js:** Download from https://nodejs.org/en/download (Versions 20.0 or higher)
   This automatically includes `npx`.

2. **Verify installation:**
   ```bash
   node --version
   npx --version
   ```

### 2. Docker/Rancher Installation

You will need container technology to build images in the later parts of this tutorial:

**macOS:**
- **Docker:** https://docs.docker.com/desktop/setup/install/mac-install/
- **Rancher Desktop** (alternative): https://docs.rancherdesktop.io/getting-started/installation/

**Windows:**
- **Docker:** https://docs.docker.com/desktop/setup/install/windows-install/
- **Rancher Desktop** (alternative): https://docs.rancherdesktop.io/getting-started/installation/


### 3. Additional Dependencies

If you haven't installed `pharia-skill` from the previous tutorial, add it now:

```bash
uv pip install pharia-skill
```

---

## Important: Terminal-Based Workflow

### Multiple Terminal Sessions Required

Unlike the previous tutorial that used Jupyter notebooks, **this section requires working directly in terminal windows**. You will often need to:

1. **Keep multiple terminal sessions open simultaneously**
2. **Run commands in separate terminal windows**
3. **Monitor different processes in parallel**

### Example Workflow

You might have:
- **Terminal 1:** Running the application server
- **Terminal 2:** Building Docker images  
- **Terminal 3:** Monitoring logs or running additional commands